In [1]:
from huggingface_hub import hf_hub_download, notebook_login
notebook_login()

In [2]:
import pickle
from google.colab import files

In [4]:
%%capture
!pip install git+https://github.com/EleutherAI/sae

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch import nn, Tensor

from functools import partial
from jaxtyping import Float, Int
from typing import Optional, Callable, Union, List, Tuple

In [7]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer

In [8]:
import gc
import pickle
import numpy as np
import matplotlib.pyplot as plt
import json
from fnmatch import fnmatch
from pathlib import Path
from typing import NamedTuple, Optional, Callable, Union, List, Tuple
# from jaxtyping import Float, Int
from collections import Counter

import einops
import torch
from torch import Tensor, nn
from huggingface_hub import snapshot_download
from natsort import natsorted
from safetensors.torch import load_model, save_model

device = "cuda" if torch.cuda.is_available() else "cpu"

In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
tokenizer.pad_token = tokenizer.eos_token

In [10]:
import json
with open('toxicity_dataset.json', 'r') as f:
  dataset = json.load(f)

In [11]:
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B")

In [12]:
import torch
import gc
from sparsify import Sae

model = model.to('cuda').eval()

def extract_and_save_sae(
    prompt: str,
    layer_id: int = 10,
    output_path: str = "sae_acts.pt"
):
    device = next(model.parameters()).device

    # 1) Tokenize + move to device
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=False)
    for k, v in inputs.items():
        inputs[k] = v.to(device)

    # 2) Forward to get hidden states
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
        hidden = outputs.hidden_states[layer_id]  # [1, seq_len, hidden_dim]

    # 3) Load SAE
    sae = Sae.load_from_hub(
        "EleutherAI/sae-Llama-3.2-1B-131k",
        hookpoint=f"layers.{layer_id}.mlp"
    ).to(device).eval()

    # 4) Encode
    with torch.no_grad():
        enc_out = sae.encode(hidden)

        # Pull out the tensor
        if isinstance(enc_out, torch.Tensor):
            sae_acts = enc_out
        elif hasattr(enc_out, "activations"):
            sae_acts = enc_out.activations
        elif isinstance(enc_out, (tuple, list)):
            sae_acts = enc_out[0]
        else:
            raise RuntimeError(f"Unrecognized encode() output: {type(enc_out)}")

    # 5) Save & return
    sae_acts_cpu = sae_acts.cpu()
    torch.save(sae_acts_cpu, output_path)
    return sae_acts_cpu


In [13]:
llama1b_acts = []
for i in range(155): #155 is the max with the free colab
  if i % 2 ==0:
    print(f"i: {i}")
  acts = extract_and_save_sae(dataset[i]['text'], 10)
  llama1b_acts.append(acts)
torch.save({'llama1b': llama1b_acts}, 'llama1b_acts.pt')

i: 0


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

i: 2


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

...

i: 152


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

i: 154


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
print(len(llama1b_acts))

155
